In [23]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy.linalg import norm
import pandas as pd
import re
import texthero as hero

/Users/jianmingyue/Library/Python/3.9/lib/python/site-packages/texthero/preprocessing.py:331: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if stem is "porter":
/Users/jianmingyue/Library/Python/3.9/lib/python/site-packages/texthero/preprocessing.py:333: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif stem is "snowball":
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jianmingyue/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz#egg=en_core_web_sm==2.3.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en_core_web_sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047086 sha256=fe0242b7ca618c968e6092099fd0f97c7f25cbf16b791f54218404655fbb8f1f
  Stored in directory: /Users/jianmingyue/Library/Caches/pip/wheels/19/d6/1c/5484b95647df5d7afaf74abde458c66c1cd427e69e801fe826
Successfully built en_core_web_sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [10]:
# Turn json into dataframe for easier data cleaning
with open("corpus/train.json") as f:
    s = f.read()
    data1 = json.loads(s)
df1 = pd.DataFrame.from_dict(data1)

with open("corpus/test.json") as f:
    s = f.read()
    data2 = json.loads(s)
df2 = pd.DataFrame.from_dict(data2)

with open("corpus/val.json") as f:
    s = f.read()
    data3 = json.loads(s)
df3 = pd.DataFrame.from_dict(data3)

df = pd.concat([df1, df2, df3])

In [11]:
df

,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...
...,...,...,...
813,13829423,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...
814,13727710,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted..."
815,13829261,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del..."
816,13680226,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n..."


In [12]:
# get people in the dialogue
def get_people(txt):
    names = re.findall(r"(?:^|\n)(\w+):", txt, re.MULTILINE)
    people = list(set(names))
    return people
df['people'] = df['dialogue'].map(get_people)

In [13]:
# Filter out data that is not equal to 2-people conversation
df1 = df.loc[df['people'].str.len() == 2]

# Checking whether is a valid operation
df1['people'].str.len().max()

# reset index
df1.reset_index(drop=True, inplace=True)

In [24]:
# Get TFIDF for each word
names = [n.lower() for n in set.union(*[set(p) for p in df1['people']])]
# vectorizer = TfidfVectorizer(ngram_range = (1, 1), token_pattern = r"\b([\w']{2,}|I)\b", stop_words = names, norm='l1')
# tfidf = vectorizer.fit_transform(df1['dialogue'])
# scores = tfidf.toarray()
# df1['tfidf'] = scores
# tfi_df = pd.DataFrame(scores, columns=vectorizer.get_feature_names_out())
# tfi_df

,00,000,00am,00pm,00s,01,0123456789,01452,01523,01632,...,ノ彡,ーー,一_一,二二二,屮ﾟдﾟ,예쁜척,ﾐθc_θ,ﾟдﾟ,ﾟдﾟщ,ﾟヮﾟ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
def modify(txt):
    tokens = re.findall(r"\b([\w']{2,}|I)\b", txt, re.MULTILINE)
    names = set(re.findall(r"(?:^|\n)(\w+):", txt, re.MULTILINE))
    return [i for i in tokens if i not in names]
df1['modify_dialogue'] = df1['dialogue'].apply(modify)

/var/folders/75/fgxvjhx11hs171zp32c86x480000gn/T/ipykernel_928/496061445.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['modify_dialogue'] = df1['dialogue'].apply(modify)


In [30]:
# df1['tfidf'] = hero.tfidf(df1['modify_dialogue'])
df1

,id,summary,dialogue,people,size,modify_dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...,"[Jerry, Amanda]",12,"[I, baked, cookies, Do, you, want, some, Sure,..."
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...,"[Olivia, Oliver]",14,"[Who, are, you, voting, for, in, this, electio..."
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...","[Tim, Kim]",87,"[Hi, what's, up, Bad, mood, tbh, I, was, going..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella....","[Edward, rachel]",23,"[Rachel, I, think, I'm, in, ove, with, Bella, ..."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...,"[Naomi, Sam]",143,"[hey, overheard, rick, say, something, don't, ..."
...,...,...,...,...,...,...
813,13829423,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...,"[Diego, Carla]",154,"[I've, got, it, what, my, date, for, graduatio..."
814,13727710,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted...","[Bev, Gita]",276,"[Hello, this, is, Beti's, Mum, I, wanted, to, ..."
815,13829261,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del...","[Julia, Robert]",213,"[Greg, just, texted, me, ugh, delete, him, alr..."
816,13680226,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n...","[Tina, Marry]",47,"[I, broke, my, nail, oh, no, know, I, have, th..."


In [17]:
def get_size(txt):
    tokens = re.findall(r"\b([\w']{2,}|I)\b", txt, re.MULTILINE)
    names = set(re.findall(r"(?:^|\n)(\w+):", txt, re.MULTILINE))
    return len([i for i in tokens if i not in names])
    
# Size of the diaglogue, names not included
df1['size'] = df1['dialogue'].map(get_size)

/var/folders/75/fgxvjhx11hs171zp32c86x480000gn/T/ipykernel_928/2594839371.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['size'] = df1['dialogue'].map(get_size)


In [35]:
df1.reset_index(drop=True, inplace=True)

In [39]:
df1['token'] = df1['modify_dialogue'].map(lambda x : ' '.join(x))


/var/folders/75/fgxvjhx11hs171zp32c86x480000gn/T/ipykernel_928/959458072.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['token'] = df1['modify_dialogue'].map(lambda x : ' '.join(x))


In [42]:
df1['tfidf'] = hero.tfidf(df1['token'])

/var/folders/75/fgxvjhx11hs171zp32c86x480000gn/T/ipykernel_928/3011831100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['tfidf'] = hero.tfidf(df1['token'])


In [43]:
df1

,id,summary,dialogue,people,size,modify_dialogue,token,tfidf
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...,"[Jerry, Amanda]",12,"[I, baked, cookies, Do, you, want, some, Sure,...",I baked cookies Do you want some Sure I'll bri...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...,"[Olivia, Oliver]",14,"[Who, are, you, voting, for, in, this, electio...",Who are you voting for in this election Libera...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...","[Tim, Kim]",87,"[Hi, what's, up, Bad, mood, tbh, I, was, going...",Hi what's up Bad mood tbh I was going to do lo...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella....","[Edward, rachel]",23,"[Rachel, I, think, I'm, in, ove, with, Bella, ...",Rachel I think I'm in ove with Bella Dont say ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...,"[Naomi, Sam]",143,"[hey, overheard, rick, say, something, don't, ...",hey overheard rick say something don't know wh...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...
11874,13829423,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...,"[Diego, Carla]",154,"[I've, got, it, what, my, date, for, graduatio...",I've got it what my date for graduation Hope y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
11875,13727710,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted...","[Bev, Gita]",276,"[Hello, this, is, Beti's, Mum, I, wanted, to, ...",Hello this is Beti's Mum I wanted to ask if yo...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
11876,13829261,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del...","[Julia, Robert]",213,"[Greg, just, texted, me, ugh, delete, him, alr...",Greg just texted me ugh delete him already He'...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
11877,13680226,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n...","[Tina, Marry]",47,"[I, broke, my, nail, oh, no, know, I, have, th...",I broke my nail oh no know I have that party t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [19]:
def size_cost(new_dialogue, pool):
    pass

In [20]:
def content_cost(new_dialogue, pool):
    pass

In [21]:
def sort_by_cost(pool, new_dialogue, w_size=0.5, w_content=0.5):
    pass